In [211]:
import requests as re
import pandas as pd
import altair as alt

In [26]:
url = "https://www.fantrax.com/fxpa/req?leagueId=962c1utjlzbm8vb8"

In [27]:
def makeRequest(gw):
    payload = {
      "msgs": [
        {
          "method": "getStandings",
          "data": {
            "leagueId": "ablch5snljylssmy",
            "timeframeType": "BY_PERIOD",
            "timeStartType": "FROM_SEASON_START",
            "period": gw
          }
        }
      ],
      "uiv": 3,
      "refUrl": f"https://www.fantrax.com/fantasy/league/962c1utjlzbm8vb8/view=REGULAR_SEASON;timeframeType=BY_PERIOD;timeStartType=FROM_SEASON_START;period={gw}",
      "dt": 2,
      "at": 0,
      "av": "3.0",
      "tz": "America/Los_Angeles",
      "v": "154.0.0"
    }
    data = re.get(url, json=payload).json()
    return data

In [37]:
max_gw = 17

In [28]:
data = makeRequest(max_gw)

In [36]:
# data

In [46]:
results_list = []
for gw in range(1,(max_gw+1)):

    data = makeRequest(gw)
    league_standings = data['responses'][0]['data']['tableList'][0]
    
    for row in league_standings['rows']:
    
        fixedCells = row['fixedCells']
        cells = row['cells']
    
        result = {
            'gw': gw,
            'rank': fixedCells[0]['content'],
            'team': fixedCells[1]['content'],
            'win': cells[0]['content'],
            'draw': cells[1]['content'],
            'loss': cells[2]['content'],
            'points': cells[3]['content'],
            'win_pct': cells[4]['content'],
            'for': cells[6]['content'],
            'against': cells[7]['content'],
            'streak': cells[8]['content'],
        }
    
        results_list.append(result)

In [47]:
df = pd.DataFrame(results_list)

In [48]:
df['for'] = df['for'].str.replace(',','')
df['against'] = df['against'].str.replace(',','')

In [49]:
clean_df = df.astype( 
    {'win': 'int64',
    'draw': 'int64',
    'loss': 'int64',
    'points': 'int64',
    'win_pct': 'float32',
    'for': 'float32',
    'against': 'float32'}
)

In [50]:
clean_df[clean_df.team == 'Seanhampton FC']

,gw,rank,team,win,draw,loss,points,win_pct,for,against,streak


In [54]:
data = clean_df

chart = alt.Chart(data).mark_line(point=True).encode(
    x = alt.X('gw', scale=alt.Scale(domain=[0, max_gw]), title="Gameweek"),
    y=alt.Y('rank:Q', scale=alt.Scale(domain=[10,1]), title="Rank"),
    color=alt.Color("team", legend=None),
)

labels = alt.Chart(data).mark_text(
    align='left', dx=5
).encode(
    x = alt.X('max(gw)', scale=alt.Scale(domain=[0, max_gw])),
    y=alt.Y('rank', aggregate={'argmax': 'gw'}, scale=alt.Scale(domain=[10,1])),
    text='team:N',
    color='team:N',
).transform_window(
    rank="rank()",
    sort=[alt.SortField("rank", order="ascending")],
    groupby=["gw"]
)
chart + labels

alt.LayerChart(...)

In [210]:
clean_df.to_csv('standings.csv', index=False)